In [ ]:
import yaml
import pandas as pd
import os
import basico

def model_find(model):
    model_directory = 'models'  # The folder where SBML model files are stored
    # First try with '_url' suffix
    sbml_path_with_url = os.path.join(model_directory, f"{model}_url.xml")
    # Then try without '_url' suffix
    sbml_path_without_url = os.path.join(model_directory, f"{model}.xml")

    # Check if the file with '_url' suffix exists
    if os.path.exists(sbml_path_with_url):
        # print(f"Loading SBML model from {sbml_path_with_url}...")
        return sbml_path_with_url

    # Check if the file without '_url' suffix exists
    if os.path.exists(sbml_path_without_url):
        # print(f"Loading SBML model from {sbml_path_without_url}...")
        return sbml_path_without_url

    # If neither file exists, raise an error
    raise FileNotFoundError(f"SBML model file not found for model '{model}'")


class DataLoader:
    def __init__(self, model):
        """Initialize with model ID. Automatically loads the corresponding YAML file."""
        self.model = model
        self.sbml_file_path = model_find(model)  # Use model_find to get the correct model file path
        self.yaml_file = self._get_yaml_file_for_model()  # Get the path to the YAML file based on model ID
        self.config = self._load_yaml_config()  # Load the YAML file's contents
        self.train_duration = self.config.get('train_duration', 100)  # Default to 100 if not specified
        self.test_duration = self.config.get('test_duration', 50)    # Default to 50 if not specified

    def _get_yaml_file_for_model(self):
        """Automatically generate the YAML file path based on the model ID."""
        yaml_directory = 'configs'  # The folder where YAML configuration files are stored
        yaml_file = f"{self.model}.yaml"  # YAML file has the same name as the model ID
        yaml_path = os.path.join(yaml_directory, yaml_file)

        # Check if the file exists in the 'configs' folder
        if not os.path.exists(yaml_path):
            raise FileNotFoundError(f"YAML file not found for model '{self.model}': {yaml_path}")
        
        return yaml_path

    def _load_yaml_config(self):
        """Load configuration from the YAML file."""
        with open(self.yaml_file, 'r') as file:
            config = yaml.safe_load(file)
        if not config:
            raise ValueError(f"YAML file {self.yaml_file} is empty or improperly formatted.")
        return config

    def run_model_for_time_period(self, duration):
        """Load model and run time course simulation for given duration."""
        # Load the SBML model
        print(f"Loading SBML model from {self.sbml_file_path}...")
        basico.load_model(self.sbml_file_path)
        
        # Run the time course simulation
        results = basico.run_time_course(duration)
        return results

    def run_model(self):
        """Run the model for both training and testing durations."""
        # Run the model for training duration
        train_results = self.run_model_for_time_period(self.train_duration)
        
        # Run the model for testing duration
        test_results = self.run_model_for_time_period(self.test_duration)
        
        return train_results, test_results

# Example usage
model = "BIOMD0000000537"  # Replace with your model ID

# Instantiate DataLoader class
data_loader = DataLoader(model)

# Run model and get results
train_results, test_results = data_loader.run_model()

# Convert results to DataFrames
train_results_df = pd.DataFrame(train_results)
test_results_df = pd.DataFrame(test_results)

# Display the results
print("Train Results:")
print(train_results_df.head())
print("Test Results:")
print(test_results_df.head())


Loading SBML model from models\BIOMD0000000537_url.xml...
Loading SBML model from models\BIOMD0000000537_url.xml...
Train Results:
      sR{serum}  sgp130{serum}  R_IL6_gp130{liver}  IL6{serum}     Ab{serum}  \
Time                                                                           
0.00   4.253507       3.900000            0.000066    0.000436  2.381820e-29   
0.06   4.253507       3.900000            0.000066    0.000436  3.588257e-29   
0.12   0.000032       3.900007            0.000066    0.000446  6.988529e+02   
0.18   0.000032       3.900043            0.000066    0.000455  6.968362e+02   
0.24   0.000032       3.900083            0.000066    0.000461  6.949118e+02   

             R  sR_IL6{gut}  sR_IL6{liver}  R_IL6_gp130{gut}  Ab_sR{serum}  \
Time                                                                         
0.00  0.438236     0.001307       0.000976          0.000084  6.104391e-26   
0.06  0.438236     0.001307       0.000976          0.000084  6.105088e-26